In [141]:
# Initial Imports 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
pd.set_option("display.max_columns", None)

# Data Read

In [142]:
df_client_data = pd.read_csv('../HET/Data/client_data.csv')
df_client_membership = pd.read_csv('../HET/Data/client_membership.csv')
df_benchmark = pd.read_csv('../HET/Data/benchmark_data.csv')

/var/folders/h4/bmfyh9q95ws8cxksqm1z9m4c0000gn/T/ipykernel_2473/1195080869.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_client_data = pd.read_csv('../HET/Data/client_data.csv')
/var/folders/h4/bmfyh9q95ws8cxksqm1z9m4c0000gn/T/ipykernel_2473/1195080869.py:3: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_benchmark = pd.read_csv('../HET/Data/benchmark_data.csv')


In [143]:
# Join Client Membership and Client Data on dw_mbr_key
df = pd.merge(df_client_membership, df_client_data, on='dw_mbr_key')

# Drop Geographic and Primary_CCS_Dx_... data 
df = df.drop(['zip_cd', 'relationship', 'MSA', 'state',
            'area_id', 'Primary_CCS_Dx_Level_1', 'Primary_CCS_Dx_Level_2'], axis=1)

In [144]:
# Converting to numerics
object_cols = df.columns[df.dtypes.eq('object')]
object_cols = object_cols.drop(['dw_mbr_key', 'gender'])

df[object_cols] = df[object_cols].apply(pd.to_numeric, errors='coerce')
df.head(10)


# Ordinal Encoding Gender 
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
df[['gender']] = encoder.fit_transform(df[['gender']])

# Paid PMPM 
Paid per Member per Month - is a primary metric used to evaluate performance.

Calculated as = (Total_Paid / Member_Months)

We'll train an ML regressor model to predict PMPM and,  
use feature importance to gain insight on which features most contribute to PMPM

## Data Manipulation

In [145]:
# Search for nuls in dataframe 
print(df.shape)

# Number of Missing Values in each column 
missing_val_count = (df.isnull().sum())
print(missing_val_count[missing_val_count > 0])

(429915, 37)
sdoh_percentile               6146
urban_percentile              6146
Total_Allowed                54617
Total_Paid                   35543
Allowed_Inpatient             3420
Allowed_Outpatient           23899
Allowed_Professional         33615
Paid_Inpatient                3363
Paid_Outpatient              16415
Paid_Professional            20864
Proc_cnt_Professional            1
Svc_unit_cnt_Professional      499
dtype: int64


**TARGET: Total_Paid** Missing approximately 12% of the data for Total_Paid 

*Allowed_Inpatient, Allowed_Outpatient, Allowed_Professional* Insurance plans payouts are typically categorical, **NA may be filled with mode**

*Paid_Inpatient, Paid_Outpatient, Paid_Professional* are respective to each member. **Cant fill, drop NA**

Rest of missing data is minimal, < 10%, **may drop NA**

In [146]:
# Drop NA 

pmpm_df = df.copy().dropna(subset=['Paid_Inpatient', 'Paid_Outpatient', 'Paid_Professional',
                                    'sdoh_percentile', 'urban_percentile', 'Svc_unit_cnt_Professional' ])


In [147]:
# Fill Allowed cols with mode
allowed_cols = ['Allowed_Inpatient', 'Allowed_Outpatient', 'Allowed_Professional', 'Total_Allowed']

for col in allowed_cols:
    pmpm_df[col].fillna(pmpm_df[col].mode()[0], inplace=True)

In [148]:
# Search for nuls in dataframe 
print(pmpm_df.shape)

# Number of Missing Values in each column 
missing_val_count = (pmpm_df.isnull().sum())
print(missing_val_count[missing_val_count > 0])

(391615, 37)
Total_Paid    2955
dtype: int64


In [149]:
# Total_Paid is now < 1%, 
# We can drop the rest 
pmpm_df = pmpm_df.dropna(subset='Total_Paid')


339949

In [150]:
# Calculate PMPM, append to dataframe 
pmpm_df['PMPM'] = pmpm_df['Total_Paid'] / pmpm_df['enrollment_month']


In [ ]:
#TODO 
# Histogram of target distribution, hue = PMPM > 0

## ML Model 

In [100]:
pmpm_df

,dw_mbr_key,gender,age,enrollment_month,risk_score,sdoh_percentile,urban_percentile,hypertension_ind,depression_ind,hyperlipidemia_ind,cad_ind,diabetes_ind,alcohol_ind,sud_ind,copd_ind,psychotic_ind,hypothyroidism_ind,anxiety_ind,cancer_ind,pregnancy_ind,annual_wellness_visit,mbr_months_ccs,Total_Allowed,Total_Paid,Allowed_Inpatient,Allowed_Outpatient,Allowed_Professional,Paid_Inpatient,Paid_Outpatient,Paid_Professional,Adm_cnt_Inpatient,Day_cnt_Inpatient,Visit_cnt_Outpatient,Visit_cnt_Professional,Proc_cnt_Professional,Svc_unit_cnt_Professional,non_utilizer,PMPM
0,"5,960,230,933",F,54,12,0.58,0.65,0.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.71429,290.65,0.0,0.0,0.0,290.65,0.0,0.0,0.0,0,0,0,1,2.0,2.0,0,0.0
1,"5,960,230,933",F,54,12,0.58,0.65,0.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.71429,100.94,0.0,0.0,0.0,100.94,0.0,0.0,0.0,0,0,0,1,1.0,1.0,0,0.0
2,"5,960,230,933",F,54,12,0.58,0.65,0.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.71429,60.27,0.0,0.0,0.0,60.27,0.0,0.0,0.0,0,0,0,1,1.0,1.0,0,0.0
3,"5,960,230,933",F,54,12,0.58,0.65,0.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.71429,100.94,0.0,0.0,0.0,100.94,0.0,0.0,0.0,0,0,0,1,1.0,1.0,0,0.0
4,"5,960,230,933",F,54,12,0.58,0.65,0.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.71429,137.34,0.0,0.0,0.0,137.34,0.0,0.0,0.0,0,0,0,0,1.0,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429910,"188,420,007,856",F,61,12,1.79,0.45,0.71,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.33333,107.62,0.0,0.0,0.0,107.62,0.0,0.0,0.0,0,0,0,0,2.0,2.0,0,0.0
429911,"188,420,007,856",F,61,12,1.79,0.45,0.71,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.33333,85.97,0.0,0.0,0.0,85.97,0.0,0.0,0.0,0,0,0,0,1.0,1.0,0,0.0
429912,"188,420,007,856",F,61,12,1.79,0.45,0.71,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.33333,41.10,0.0,0.0,0.0,41.10,0.0,0.0,0.0,0,0,0,1,2.0,2.0,0,0.0
429913,"188,420,007,856",F,61,12,1.79,0.45,0.71,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.33333,516.69,0.0,0.0,0.0,516.69,0.0,0.0,0.0,0,0,0,3,6.0,6.0,0,0.0
